## Exploratory analysis of video data from Python session 

(E and K pair debugging an implementation of the hangman game)

In [1]:
# Import software libraries
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
import cv2

import seaborn as sns # for the nicer default plot aesthetics 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# % matplotlib inline 
% matplotlib notebook 
# ^ with notebook, remember to initiate and work on figures in the object-oriented way...

from IPython.display import display
pd.options.display.max_columns = None
import pprint as pp

import helpers # written by us for MMLA pipeline purposes

#### Pre-processing the front-facing camera video with ffmpeg and opencv

In [2]:
# Do this in a terminal window.

# [] Resize images with ffmpeg
# https://trac.ffmpeg.org/wiki/Scaling
# $ ffmpeg -i screenCapture_8to11.avi -vf scale=320:-1 screenCapture_8to11.avi
# $ ffmpeg -i webcam.avi -vf scale=320:-1 webcam_320.avi

# [] Convert if needed
# https://www.labnol.org/internet/useful-ffmpeg-commands/28490/
# $ ffmpeg -i screenCapture_8to11.avi -c:v libx264 screenCapture_8to11.mp4

# [] Extract all the frames!
# https://stackoverflow.com/questions/10957412/fastest-way-to-extract-frames-using-ffmpeg
# https://trac.ffmpeg.org/wiki/Seeking
# $ ffmpeg -i webcam_segment_320.avi -r 1/1 $filename%03d.jpg
# $ ffmpeg -r 1 webcam_segment_320.avi -r 1 "$filename%03d.png"
# $ ffmpeg -i webcam_segment_320.avi "$filename%03d.png"

# https://stackoverflow.com/questions/34786669/extract-all-video-frames-as-images-with-ffmpeg/34786929
# $ mkdir frames
# $ ffmpeg -i "webcam_segment_320.avi" "frames/frame_%03d.jpg"
# $ ffmpeg -i "webcam.avi" "frames/frame_%03d.jpg"

In [3]:
# Yeah um this doesn't work off the shelf in jupyter notebooks
# # https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html

# cap = cv2.VideoCapture('data\webcam_segment_320.avi')

# while(cap.isOpened()):
#     ret, frame = cap.read()
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     pp.pprint(gray)
# #     cv2.imshow('frame',gray)
# #     if cv2.waitKey(1) & 0xFF == ord('q'):
# #         print('break')
# #         break

# cap.release()
# cv2.destroyAllWindows()

In [4]:
# Do you want to run this cell when you run the entire notebook?
# (This code only needs to be run once for a given session.)
run_image_split = False

# Do this in a terminal window if you haven't already
# $ mkdir frames
# $ mkdir split_frames

# Extract the frames as mentioned in a previous cell 

# Look at your frames subfolder and manually set this. can automate later if desired
NUM_FRAMES = 24254

# Let's split
if run_image_split:
    for i in range(1, NUM_FRAMES + 1):
        # Find and open a frame image
        if i<10:
            filename_i = '00' + str(i)
        elif i<100:
            filename_i = '0' + str(i)
        else:
            filename_i = str(i)
        try:
            img = cv2.imread('data/frames/frame_{filename_i}.jpg'.format(filename_i=filename_i))
            # pp.pprint(img)
        except:
            print('except... please debug!')
            break

        # Split
        rows, cols = img.shape[0], img.shape[1]
        # print(rows, cols)

        # top left corner 0, 0
        # top right corner rows, 0
        # bottom left corner 0, cols
        # bottom right corner rows, cols
        lhs_img = img[0:rows, :cols/2]
        rhs_img = img[0:rows, cols/2:]

        # Save
        cv2.imwrite('data/split_frames/{filename_i}_lhs.png'.format(filename_i=filename_i),lhs_img)
        cv2.imwrite('data/split_frames/{filename_i}_rhs.png'.format(filename_i=filename_i),rhs_img)
        
    print('done ;)')
print('did not run')

did not run


In [5]:
# Do you want to run this cell when you run the entire notebook?
run_image_rotate = False

if run_image_rotate:
    for i in range(1, NUM_FRAMES + 1):
        # Find and open a frame image
        if i<10:
            filename_i = '00' + str(i)
        elif i<100:
            filename_i = '0' + str(i)
        else:
            filename_i = str(i)
        try:
            lhs_img = cv2.imread('data/upside_down_split_frames/{filename_i}_lhs.png'.format(filename_i=filename_i))
            rows,cols = lhs_img.shape[0], lhs_img.shape[1]
            M = cv2.getRotationMatrix2D((cols/2,rows/2),180,1)
            dst = cv2.warpAffine(lhs_img,M,(cols,rows))            
            cv2.imwrite('data/ready_frames/{filename_i}_lhs.png'.format(filename_i=filename_i),dst)            

            rhs_img = cv2.imread('data/upside_down_split_frames/{filename_i}_rhs.png'.format(filename_i=filename_i))
            rows,cols = rhs_img.shape[0], lhs_img.shape[1]
            M = cv2.getRotationMatrix2D((cols/2,rows/2),180,1)
            dst = cv2.warpAffine(rhs_img,M,(cols,rows))            
            cv2.imwrite('data/ready_frames/{filename_i}_rhs.png'.format(filename_i=filename_i),dst)            
    
        except:
            print('except... please debug!')
            break
        print('done ;)')
print('did not run')

done ;)


In [6]:
# https://trac.ffmpeg.org/wiki/Slideshow

# $ ffmpeg -i '%03d_lhs.png' lhs_KBR.mp4
# $ ffmpeg -i '%03d_rhs.png' rhs_EQW.mp4